<a href="https://colab.research.google.com/github/Kannanitin/Kannanitin/blob/main/demo/vibevoice_realtime_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VibeVoice-Realtime Colab — T4 Quickstart



## Step 1: Setup Environment

In [1]:
# Check for T4 GPU
import torch
if torch.cuda.is_available() and "T4" in torch.cuda.get_device_name(0):
    print("✅ T4 GPU detected")
else:
    print("""
    ⚠️ WARNING: T4 GPU not detected

    The recommended runtime for this Colab notebook is "T4 GPU".

    To change the runtime type:

        1. Click on "Runtime" in the top navigation menu
        2. Click on "Change runtime type"
        3. Select "T4 GPU"
        4. Click "OK" if a "Disconnect and delete runtime" window appears
        5. Click on "Save"

    """)

# Clone the VibeVoice repository
![ -d /content/VibeVoice ] || git clone --quiet --branch main --depth 1 https://github.com/microsoft/VibeVoice.git /content/VibeVoice
print("✅ Cloned VibeVoice repository")

# Install project dependencies
!uv pip --quiet install --system -e /content/VibeVoice
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared && chmod +x cloudflared
print("✅ Installed dependencies")

# Download model
from huggingface_hub import snapshot_download
snapshot_download("microsoft/VibeVoice-Realtime-0.5B", local_dir="/content/models/VibeVoice-Realtime-0.5B")
print("✅ Downloaded model: microsoft/VibeVoice-Realtime-0.5B")


✅ T4 GPU detected
✅ Cloned VibeVoice repository
✅ Installed dependencies


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.04G [00:00<?, ?B/s]

figures/Fig1.png:   0%|          | 0.00/124k [00:00<?, ?B/s]

✅ Downloaded model: microsoft/VibeVoice-Realtime-0.5B


[Optional] If the download exceeds 1 minute, it is probably stuck. You can: (1) interrupt the execution, (2) log in to Hugging Face, and (3) try download again.

In [2]:
from huggingface_hub import login
login()

In [ ]:
snapshot_download("microsoft/VibeVoice-Realtime-0.5B", local_dir="/content/models/VibeVoice-Realtime-0.5B")
print("✅ Downloaded model: microsoft/VibeVoice-Realtime-0.5B")

## Step 2: Launch VibeVoice-Realtime Demo

In [3]:
import subprocess, re, time, threading

srv = subprocess.Popen(
    "python /content/VibeVoice/demo/vibevoice_realtime_demo.py --model_path /content/models/VibeVoice-Realtime-0.5B --port 8000",
    shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True,
)
cf = subprocess.Popen(
    "./cloudflared tunnel --url http://localhost:8000 --no-autoupdate",
    shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True,
)

public_url = None
server_ready = False
url_pattern  = re.compile(r"(https://[a-z0-9-]+\.trycloudflare\.com)")

def read_srv():
    global server_ready
    for ln in srv.stdout:
        print(ln.strip())
        if "Uvicorn running on" in ln:
            server_ready = True

def read_cf():
    global public_url
    for ln in cf.stdout:
        m = url_pattern.search(ln)
        if m:
            public_url = m.group(1)
            break

threading.Thread(target=read_srv, daemon=True).start()
threading.Thread(target=read_cf,  daemon=True).start()


while True:
    if server_ready and public_url:
        print(f"✅ Public URL: {public_url}\n");
        public_url = None
    time.sleep(0.25)

2025-12-14 01:34:52.135803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765676092.152653    1641 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765676092.158874    1641 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765676092.171948    1641 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765676092.171967    1641 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765676092.171970    1641 computation_placer.cc:177] computation placer alr

KeyboardInterrupt: 